<a href="https://colab.research.google.com/github/ersanjeevkm/amazon-esci-product-embeddings/blob/main/notebooks/Qdrant_upsert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
dataset_path = kagglehub.dataset_download('sanjeevkm/amazon-esci-product')

100%|██████████| 1.23G/1.23G [00:13<00:00, 96.8MB/s]

Extracting files...


# Load product details and context data

In [ ]:
import pandas as pd

df_products = pd.read_parquet(dataset_path + '/products_details.parquet')

In [ ]:
!pip install -q "qdrant-client[fastembed]>=1.14.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient, models
from google.colab import userdata

qdrant_key = userdata.get("qdrant_key")

qd_client = QdrantClient(
    url="https://4dd19404-3386-4f5c-9a63-a3034166e3b9.eu-west-2-0.aws.cloud.qdrant.io",
    api_key=qdrant_key
)

In [ ]:
qd_client.delete_collection('esci_products_multi_vec')

True

In [ ]:
qd_client.get_collections()

CollectionsResponse(collections=[])

# Upsert

**Symantic search**
We will use embedding dimensionality of 512 and english language encoder

**Keyword search**
Will be using bm25 (best match 25)

In [ ]:
EMBEDDING_DIMENSIONALITY = 512
collection_name = "esci_products_multi_vec"

In [ ]:
# Create the collection with both vector types
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [ ]:
df_products['context'] = (
    "Product Title:" + df_products['product_title'].astype(str) +
    "\nProduct Description:" + df_products['product_description'].astype(str) +
    "\nProduct Bullet Points:" + df_products['product_bullet_point'].astype(str) +
    "\nProduct Brand:" + df_products['product_brand'].astype(str) +
    "\nProduct Color:" + df_products['product_color'].astype(str)
)

In [ ]:
import uuid

points = [
    models.PointStruct(
        id=uuid.uuid4().hex,
        vector={
            "jina-small": models.Document(
                text=product["context"],
                model="jinaai/jina-embeddings-v2-small-en",
            ),
            "bm25": models.Document(
                text=product["context"],
                model="Qdrant/bm25",
            ),
        },
        payload={
            'product_id': product['product_id'],
            'product_title': product['product_title'],
            'product_description': product['product_description'],
            'product_bullet_point': product['product_bullet_point'],
            'product_brand': product['product_brand'],
            'product_color': product['product_color']
        }
    )
    for _, product in df_products.iterrows()
]

In [ ]:
from tqdm import tqdm

BATCH_SIZE = 200

for i in tqdm(range(0, len(points), BATCH_SIZE)):
    batch = points[i:i + BATCH_SIZE]
    qd_client.upsert(
        collection_name=collection_name,
        points=batch
    )


  0%|          | 0/6080 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

...

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

  0%|          | 3/6080 [06:20<213:50:38, 126.68s/it]


KeyboardInterrupt: 

# Symantic search (cosine similarity btw dense vectors)

# Hybrid search (Symantic and RRF)
Prefetching 5x and reranking using RRF (Reciprocal Rank Fusion)
$$
\text{Score}(d) = \sum_{i=1}^{n} \frac{1}{k + \text{rank}_i(d)}
$$